Clase Nivel Avanzado: Análisis Temporal, Funciones Personalizadas y Visualizaciones Sofisticadas en Datos de Tenis con Pandas
Objetivo: Que el estudiante sea capaz de realizar análisis temporales básicos, aplicar funciones personalizadas complejas al DataFrame y crear visualizaciones avanzadas para comunicar hallazgos de manera efectiva en el contexto del análisis de datos deportivos.

Duración: 1 hora y 30 minutos

Dataset: El mismo dataset atp_tennis.csv.

Estructura de la Clase:

Repaso y Preparación Avanzada del Dataset (15 minutos):

Recordatorio de los conceptos de las clases anteriores.
Carga del dataset, limpieza básica (eliminación de NaNs y conversión de 'Date' a datetime).
Establecimiento de la columna 'Date' como índice del DataFrame para facilitar el análisis temporal.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Cargamos el dataset
df = pd.read_csv('atp_tennis.csv')

# Limpieza básica: eliminar filas con valores nulos
df_cleaned = df.dropna().copy()

# Convertimos la columna 'Date' a datetime
df_cleaned['Date'] = pd.to_datetime(df_cleaned['Date'])

# Establecemos la columna 'Date' como índice
df_indexed = df_cleaned.set_index('Date').sort_index()

print("Dataset con 'Date' como índice:")
print(df_indexed.head())

.set_index('Date'): Convierte la columna 'Date' en el índice del DataFrame.
.sort_index(): Ordena el DataFrame por su índice (en este caso, la fecha), lo cual es crucial para el análisis temporal.
Análisis Temporal Básico (25 minutos):

Tendencias anuales en el número de partidos:

In [ ]:
# Resampleamos los datos por año y contamos el número de partidos
yearly_matches = df_indexed.resample('Y').size()
print("\nNúmero de partidos por año:")
print(yearly_matches)

# Visualizamos la tendencia anual
plt.figure(figsize=(12, 6))
yearly_matches.plot(kind='line', marker='o')
plt.title('Tendencia Anual del Número de Partidos Jugados')
plt.xlabel('Año')
plt.ylabel('Número de Partidos')
plt.grid(True)
plt.show()

.resample('Y'): Realiza un remuestreo de los datos basado en la frecuencia 'Y' (anual). Como tenemos la fecha como índice, esto agrupa los datos por año.
.size(): Cuenta el número de filas en cada grupo anual, lo que representa el número de partidos jugados en ese año.
.plot(kind='line', marker='o'): Crea un gráfico de líneas con marcadores en los puntos de datos para visualizar la tendencia.
plt.grid(True): Añade una cuadrícula al gráfico para facilitar la lectura de los valores.
Análisis de la frecuencia de torneos a lo largo del tiempo:

In [ ]:
# Contamos el número de veces que se jugó cada torneo por año
yearly_tournament_counts = df_indexed.groupby(pd.Grouper(freq='Y'))['Tournament'].value_counts().unstack(fill_value=0)
print("\nNúmero de veces que se jugó cada torneo por año (primeras columnas):")
print(yearly_tournament_counts.iloc[:, :5].head()) # Mostramos solo las primeras 5 columnas para no saturar la salida

# Visualizamos la tendencia de algunos torneos importantes (ejemplo: Wimbledon, Australian Open)
plt.figure(figsize=(12, 6))
if 'Wimbledon' in yearly_tournament_counts.columns:
    yearly_tournament_counts['Wimbledon'].plot(label='Wimbledon', marker='o')
if 'Australian Open' in yearly_tournament_counts.columns:
    yearly_tournament_counts['Australian Open'].plot(label='Australian Open', marker='o')
plt.title('Frecuencia Anual de Torneos Seleccionados')
plt.xlabel('Año')
plt.ylabel('Número de Veces Jugado')
plt.legend()
plt.grid(True)
plt.show()

pd.Grouper(freq='Y'): Se utiliza dentro de groupby() cuando el índice es una fecha para agrupar por períodos de tiempo específicos (en este caso, años).
.value_counts().unstack(fill_value=0): Cuenta la frecuencia de cada torneo dentro de cada grupo anual y luego utiliza unstack() para pivotar la tabla, convirtiendo los nombres de los torneos en columnas y llenando los valores faltantes con 0.
.iloc[:, :5]: Permite seleccionar un subconjunto de las columnas del DataFrame (en este caso, las primeras 5).
plt.legend(): Muestra la leyenda del gráfico para identificar las líneas de los diferentes torneos.
Aplicación de Funciones Personalizadas Complejas (30 minutos):

Función para calcular una "puntuación de sorpresa" basada en la diferencia de ranking y las cuotas (esto es un ejemplo y puede requerir más lógica del negocio del tenis):

In [ ]:
import numpy as np

def calculate_surprise_score(row):
    rank_diff = abs(row['Rank_1'] - row['Rank_2'])
    if pd.notna(row['Odd_1']) and pd.notna(row['Odd_2']):
        # Cuanto mayor sea la cuota del ganador, mayor la sorpresa (simplificado)
        if row['Winner'] == row['Player_1']:
            surprise = rank_diff + (row['Odd_1'] - 1)
        else:
            surprise = rank_diff + (row['Odd_2'] - 1)
        return surprise
    return None

df_indexed['Surprise_Score'] = df_indexed.apply(calculate_surprise_score, axis=1)
print("\nPrimeras filas con la 'Surprise_Score':")
print(df_indexed[['Rank_1', 'Rank_2', 'Odd_1', 'Odd_2', 'Winner', 'Surprise_Score']].head())

# Visualizamos la distribución de la puntuación de sorpresa
plt.figure(figsize=(10, 6))
sns.histplot(df_indexed['Surprise_Score'].dropna(), bins=30, kde=True)
plt.title('Distribución de la Puntuación de Sorpresa')
plt.xlabel('Puntuación de Sorpresa')
plt.ylabel('Frecuencia')
plt.show()

import numpy as np: Importa la librería NumPy, que a menudo se utiliza con Pandas para operaciones numéricas.
pd.notna(): Función de Pandas que verifica si un valor no es NaN (Not a Number).
La función calculate_surprise_score es un ejemplo de cómo se podría crear una métrica personalizada combinando diferentes columnas. La lógica específica dependerá del conocimiento del deporte.
Función para identificar rachas de victorias de un jugador (esto requeriría un procesamiento más complejo y podría ser un punto de partida para un proyecto más grande):

In [ ]:
def identify_streaks(df, player_name):
    player_matches = ((df['Player_1'] == player_name) & (df['Winner'] == player_name)) | ((df['Player_2'] == player_name) & (df['Winner'] == player_name))
    streaks = []
    current_streak = 0
    for is_win in player_matches.sort_index(): # Asegurarse de que esté ordenado por fecha
        if is_win:
            current_streak += 1
        else:
            if current_streak > 0:
                streaks.append(current_streak)
            current_streak = 0
    if current_streak > 0:
        streaks.append(current_streak)
    return streaks

federer_streaks = identify_streaks(df_indexed.copy(), 'Roger Federer') # Usamos .copy() para no modificar el original
if federer_streaks:
    print(f"\nRachas de victorias de Roger Federer: {federer_streaks}")
    print(f"Racha más larga de Roger Federer: {max(federer_streaks) if federer_streaks else 0}")
else:
    print("\nNo se encontraron rachas de victorias para Roger Federer.")

Esta función es un ejemplo más avanzado que itera sobre las filas (después de filtrar los partidos del jugador y ordenarlos por fecha) para identificar secuencias de victorias. Es importante notar que este tipo de análisis secuencial puede ser más eficiente con otras técnicas o librerías para datasets muy grandes.
Visualizaciones Sofisticadas (15 minutos):

Gráfico de dispersión de la relación entre la diferencia de ranking y la "sorpresa":

In [ ]:
plt.figure(figsize=(10, 7))
sns.scatterplot(x='Rank_Difference', y='Surprise_Score', data=df_indexed.dropna())
plt.title('Relación entre Diferencia de Ranking y Puntuación de Sorpresa')
plt.xlabel('Diferencia de Ranking')
plt.ylabel('Puntuación de Sorpresa')
plt.grid(True)
plt.show()

sns.scatterplot(...): Crea un gráfico de dispersión para visualizar la relación entre dos variables numéricas.
(Opcional, si hay tiempo e interés): Introducción a visualizaciones interactivas con Plotly Express (requiere instalación: pip install plotly)

In [ ]:
# import plotly.express as px

# top_tournaments = df_indexed['Tournament'].value_counts().nlargest(10).index
# filtered_df = df_indexed[df_indexed['Tournament'].isin(top_tournaments)]

# fig = px.box(filtered_df, x='Tournament', y='Rank_Difference', title='Distribución de la Diferencia de Ranking en los 10 Torneos Principales')
# fig.show()

import plotly.express as px: Importa la librería Plotly Express, que facilita la creación de visualizaciones interactivas.
Las visualizaciones interactivas permiten al usuario interactuar con el gráfico (zoom, hover para ver detalles, etc.).
Ejercicios para el Estudiante (10 minutos):

Realiza un análisis temporal del número de partidos jugados por superficie a lo largo de los años. ¿Hay alguna tendencia notable?
Crea una función que calcule la diferencia en los puntos del ranking ('Pts_1' - 'Pts_2'). Aplica esta función al DataFrame y visualiza su distribución.
Intenta identificar los torneos que históricamente han tenido las mayores "sorpresas" promedio (basado en la métrica que creamos).
(Desafío) Investiga cómo podrías utilizar ventanas móviles (rolling windows) en Pandas para analizar tendencias en el ranking promedio de los ganadores a lo largo del tiempo.
Cierre (5 minutos):

Recapitulación de las técnicas avanzadas aprendidas.
Discusión sobre la importancia del análisis temporal y la creación de métricas personalizadas en el análisis deportivo.
Menciones sobre posibles caminos a seguir en el aprendizaje (series de tiempo más avanzadas, machine learning para predicción deportiva, etc.).
Este nivel avanzado busca consolidar las habilidades del estudiante y abrirle las puertas a análisis más profundos y personalizados de datos deportivos. ¡Espero que esta clase sea un gran avance para tu alumno!